In [1]:
import os
import gc
import numpy as np
import pandas as pd
import joblib
from datetime import datetime

from lightgbm import LGBMClassifier
import optuna
from prunedcv import PrunedCV

from codes.utils import import_data, cross_val_score_auc, reduce_mem_usage, fix_dtypes
from codes.fe_browser import latest
from codes.fe_emails import proton, mappings
from codes.fe_cards import stats
from codes.fe_date import dates
from codes.fe_relatives import divisions, divisions_float
from codes.fe_categorical import pairs, wtf, cat_limit, encode_cat
from codes.prepro import prepro
from codes.fe_users import users_stats

from tqdm import tqdm_notebook

from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.base import TransformerMixin

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
DATA_PATH = '../input/'

In [8]:
if os.path.isfile('features_train.pkl'):
    X_train_cols1 = pd.read_csv('../input/train_identity.csv', nrows=5, index_col=0)
    X_train_cols2 = pd.read_csv('../input/train_transaction.csv', nrows=5, index_col=0)
    X_train = joblib.load('features_train.pkl')
    X_test = joblib.load('features_test.pkl')
    X_train_ext = joblib.load('X_train_ext.pkl')
    X_test_ext = joblib.load('X_test_ext.pkl')
    y_train = joblib.load('y_train.pkl')
    
else:

    print('fix_dtypes')
    X_train, X_test = fix_dtypes(X_train, X_test)
    print('users_stats')
    X_train, X_test = users_stats(X_train, X_test)
    print('latest')
    X_train, X_test = latest(X_train, X_test)
    print('proton')
    X_train, X_test = proton(X_train, X_test)
    print('nulls1')
    X_train['nulls1'] = X_train.isna().sum(axis=1)
    X_test['nulls1'] = X_test.isna().sum(axis=1)
    print('mappings')
    X_train, X_test = mappings(X_train, X_test)
    print('stats')
    X_train, X_test = stats(X_train, X_test)
    print('divisions')
    X_train, X_test = divisions(X_train, X_test)
    print('dates')
    X_train, X_test = dates(X_train, X_test)
    print('pairs')
    X_train, X_test = pairs(X_train, X_test)
    print('encode_cat')
    X_train, X_test = encode_cat(X_train, X_test)
    print('wtf')
    X_train, X_test = wtf(X_train, X_test)
    print('y')
    y_train = X_train['isFraud'].copy()
    X_train = X_train.drop('isFraud', axis=1)
    print('divisions_float')
    X_train, X_test = divisions_float(X_train, X_test)
    print('prepro')
    X_train, X_test = prepro(X_train, X_test)
    print('reduce_mem_usage')
    # X_train = reduce_mem_usage(X_train)
    # X_test = reduce_mem_usage(X_test)
    print('np.inf')
    X_train[X_train == np.inf] = -999
    X_train[X_train == -np.inf] = -999
    X_train[X_train.isna()] = -999
    X_test[X_test == np.inf] = -999
    X_test[X_test == -np.inf] = -999
    X_test[X_test.isna()] = -999
    print('TransactionDT')
    X_test.drop(['TransactionDT'], axis=1, inplace=True)
    X_train.drop(['TransactionDT'], axis=1, inplace=True)
    joblib.dump(X_train, 'features_train.pkl')
    joblib.dump(X_test, 'features_test.pkl')
    joblib.dump(y_train, 'y_train.pkl')

In [4]:
all_columns = list(X_train.columns)

columns_num_base = list(set(
    ['TransactionAmt',
     'user_trx_cnt',
     'nulls1',
     'dist1',
     'dist2'] +
    ['C{}'.format(i) for i in range(1,15)] \
    + ['D{}'.format(i) for i in range(1,16)] \
    + ['V' + str(i) for i in range(1,340)]))

num_cols = []
for col in all_columns:
    for col2 in columns_num_base:
        if col2 in col:
            num_cols.append(col)
            
spec_nums = ['_count_', 
             '_mean_', 
             '_std_', 
             'Transaction_day_of_week',
             'Transaction_hour_of_day']
spec_nums_cols = []
for col in all_columns:
    for col2 in spec_nums:
        if col2 in col:
            spec_nums_cols.append(col)

num_cols = list(set(num_cols + columns_num_base + spec_nums_cols))
cat_cols = [col for col in all_columns if col not in num_cols]

In [5]:
num_cols_fin = [col for col in num_cols if col in list(X_train.columns)]
cat_cols_fin = [col for col in cat_cols if col in list(X_train.columns)]
# cat_cols_fin.remove('screen_width')
# cat_cols_fin.remove('screen_height')
# cat_cols_fin.remove('screen_ratio')

In [6]:
X_train.head()

,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C9,C11,C12,C13,C14,D1,D2,D3,D5,D10,D12,D13,D14,M1,M2,M3,M4,M5,M6,M7,M8,M9,V3,V5,V7,V8,V12,V13,V19,V20,V21,V23,V24,V25,V26,V27,V30,V32,V33,V34,V35,V36,V37,V38,V41,V43,V44,V45,V46,V47,V49,V52,V53,V54,V55,V56,V58,V61,V62,V63,V66,V67,V68,V70,V72,V73,V74,V75,V76,V77,V78,V81,V82,V83,V86,V87,V88,V90,V94,V95,V96,V97,V98,V99,V100,V101,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V127,V130,V133,V134,V172,V245,V275,V280,V281,V282,V283,V285,V286,V289,V290,V291,V292,V294,V296,V300,V301,V304,V305,V306,V307,V308,V309,V310,V312,V313,V314,V316,V318,V319,V320,V321,V322,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,user_trx_cnt,user_TransactionAmt_mean,user_TransactionAmt_std,latest_browser,device_version,os_id_30,version_id_30,browser_id_31,version_id_31,screen_width,screen_height,screen_ratio,device_name,P_isproton,R_isproton,nulls1,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix,card1_count_full,card2_count_full,card3_count_full,card4_count_full,card5_count_full,card6_count_full,addr1_count_full,addr2_count_full,TransactionAmt_to_mean_card1,TransactionAmt_to_mean_card4,TransactionAmt_to_std_card1,id_02_to_mean_card4,id_02_to_std_card1,D15_to_mean_card1,D15_to_std_card1,D15_to_std_card4,D15_to_std_addr1,Transaction_day_of_week,Transaction_hour_of_day,TransactionAmt_decimal,special_hour_day,special_day_week,id_02__id_20,id_02__D8,D11__device_name,device_name__P_emaildomain,P_emaildomain__C2,card2__dist1,card1__card5,card2__id_20,card3__card5,P_emaildomain_bin__R_emaildomain_bin,P_emaildomain_suffix__R_emaildomain_suffix,card5__P_emaildomain,addr1__card1,addr2__card1,addr1__card4,id_01_count_dist,id_31_count_dist,id_33_count_dist,id_35_count_full,V96_trx,C4_trx,V304_trx,V44_trx,V30_trx,V310_trx,V61_trx,V280_trx,V283_trx,V99_trx,C13_trx,V124_trx,V5_trx,V170_trx,C5_trx,V125_trx,V75_trx,V292_trx,V49_trx,V127_trx,V285_trx,V13_trx,V307_trx,V282_trx,V82_trx,V54_trx,V11_trx,V56_trx,V290_trx,V87_trx,D2_trx,V53_trx,V3_trx,V70_trx,C14_trx,C9_trx,V262_trx,V26_trx,V12_trx,V23_trx,V83_trx,V90_trx,D9_trx,V77_trx,C11_trx,C1_trx,V20_trx,D15_trx,V67_trx,D11_trx,V36_trx,V62_trx,V291_trx,V35_trx,C6_trx,D3_trx,V76_trx,D1_trx,C2_trx,D10_trx,V19_trx,C1__div__C2,C1__div__C3,C1__div__C4,C1__div__C5,C1__div__C6,C1__div__C9,C1__div__C11,C1__div__C13,C1__div__C14,C2__div__C5,C2__div__C6,C2__div__C9,C2__div__C11,C2__div__C13,C2__div__C14,C2__div__D2,C2__div__D4,C2__div__D11,C2__div__D15,C3__div__C5,C4__div__C7,C4__div__C10,C4__div__D8,C4__div__D9,C5__div__C6,C5__div__C8,C5__div__C9,C5__div__C11,C5__div__D6,C5__div__D7,C6__div__C9,C6__div__C11,C6__div__C12,C6__div__C13,C6__div__C14,C6__div__D1,C6__div__D2,C6__div__D3,C6__div__D15,C7__div__C8,C7__div__C9,C8__div__C13,C9__div__C11,C9__div__C13,C9__div__C14,C9__div__D1,C9__div__D2,C9__div__D3,C9__div__D4,C9__div__D5,C9__div__D10,C9__div__D15,C10__div__D11,C10__div__D13,C11__div__C13,C11__div__C14,C11__div__D1,C11__div__D14,C11__div__D15,C13__div__C14,C13__div__D1,C13__div__D2,C13__div__D3,C13__div__D4,C13__div__D10,C13__div__D11,C13__div__D15,C14__div__D1,C14__div__D5,C14__div__D10,C14__div__D15,D1__div__D2,D1__div__D3,D1__div__D4,D1__div__D5,D1__div__D10,D1__div__D11,D1__div__D15,D2__div__D3,D2__div__D4,D2__div__D5,D2__div__D6,D2__div__D8,D2__div__D9,D2__div__D10,D2__div__D11,D2__div__D13,D2__div__D14,D2__div__D15,D3__div__D4,D3__div__D5,D3__div__D7,D3__div__D8,D3__div__D9,D3__div__D10,D3__div__D11,D3__div__D14,D3__div__D15,D4__div__D5,D4__div__D6,D4__div__D8,D4__div__D9,D4__div__D10,D4__div__D11,D4__div__D12,D4__div__D13,D4__div__D14,D4__div__D15,D5__div__D7,D5__div__D8,D5__div__D11,D5__div__D13,D5__div__D14,D5__div__D1

In [9]:
cols_init = list(set(list(X_train_cols1.columns) + list(X_train_cols2.columns)))

In [10]:
cols_new = []
for col in list(X_test_ext.columns):
    if col not in cols_init:
        cols_new.append(col)

In [11]:
cols_new

['hour',
 'weekday',
 'ProductCD_count',
 'card1_count',
 'card2_count',
 'card3_count',
 'card4_count',
 'card5_count',
 'card6_count',
 'addr1_count',
 'addr2_count',
 'C1_count',
 'C2_count',
 'C3_count',
 'C4_count',
 'C5_count',
 'C6_count',
 'C7_count',
 'C8_count',
 'C9_count',
 'C10_count',
 'C11_count',
 'C12_count',
 'C13_count',
 'C14_count',
 'card1_amtcount',
 'card1_C5count',
 'card2_amtcount',
 'card2_C5count',
 'card5_amtcount',
 'card5_C5count',
 'addr1_amtcount',
 'addr1_C5count',
 'addr2_amtcount',
 'addr2_C5count']